In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

In [ ]:
# reading in dataset
df_og = pd.read_csv('data/train.csv')
df_og.drop('id', axis=1, inplace=True)
df_og

In [ ]:
# check for missing values
df_og.isnull().sum()

In [ ]:
# Columns = 'Product ID', 'Type', 'Air temperature [K]', 'Process temperature [K]',
#        'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]',
#        'Machine failure', 'TWF', 'HDF', 'PWF', 'OSF', 'RNF'

# 
# Feature engineering
# 

# Use one-hot encoding for categorical variables
# pd.get_dummies converts categorical variables into dummy/indicator variables
# We want to use one-hot encoding only for the "Type" column and no other columns
# so we'll use pd.get_dummies only on the "Type" column and add the new columns to the dataframe
df_onehot = pd.get_dummies(df_og, columns=['Type'])
df_onehot


In [ ]:
# create variable to store the difference between process temperature and air temperature
df_onehot['temp_difference'] =   df_onehot['Process temperature [K]'] - df_onehot['Air temperature [K]']
df_onehot

# we will re-add product IDs that are over a certain threshold
threshold = 20

# get product IDs that are over a certain threshold
product_ids = df_og['Product ID'].value_counts()
# print(product_ids)
ids_to_keep = product_ids[product_ids > threshold].index

# keep the product ID if it is in the list of IDs to keep, else replace with 'other'
df_onehot['Product ID'] = df_og['Product ID'].apply(lambda x: x if x in ids_to_keep else 'other')
df_onehot

In [ ]:
df_onehot

In [ ]:
# split into train and test
for train_index, val_index in KFold(n_splits=5, shuffle=True, random_state=42).split(df):
    train = df.iloc[train_index]
    val = df.iloc[val_index]

    
    
    
    



In [ ]:
#look for imbalance
df['Machine failure'].value_counts()

